In [1]:
from scipy.io import loadmat
imagelabels = loadmat('imagelabels.mat')
print(imagelabels)

setid = loadmat('setid.mat')
print(setid)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNX86, Created on: Thu Feb 19 15:43:33 2009', '__version__': '1.0', '__globals__': [], 'labels': array([[77, 77, 77, ..., 62, 62, 62]], dtype=uint8)}
{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNX86, Created on: Thu Feb 19 17:38:58 2009', '__version__': '1.0', '__globals__': [], 'trnid': array([[6765, 6755, 6768, ..., 8026, 8036, 8041]], dtype=uint16), 'valid': array([[6773, 6767, 6739, ..., 8028, 8008, 8030]], dtype=uint16), 'tstid': array([[6734, 6735, 6737, ..., 8044, 8045, 8047]], dtype=uint16)}


In [2]:
import pandas as pd

df_labels = pd.DataFrame(imagelabels['labels'])
df_labels = df_labels.T
# rename the column such that it is image_id and labels
df_labels['image_id'] = df_labels.index +1 
# drop index by making image_id as index
df_labels.set_index('image_id', inplace=True)
# set headers to be true
df_labels.columns = ['labels']
df_labels.head()


,labels
image_id,
1,77
2,77
3,77
4,77
5,77


In [3]:
df_setid_trnid = pd.DataFrame(setid['trnid'])
df_setid_valid = pd.DataFrame(setid['valid'])
df_setid_tstid = pd.DataFrame(setid['tstid'])

df_setid_trnid = df_setid_trnid.T
df_setid_valid = df_setid_valid.T
df_setid_tstid = df_setid_tstid.T

print(df_setid_trnid.shape)
print(df_setid_valid.shape)
print(df_setid_tstid.shape)

(1020, 1)
(1020, 1)
(6149, 1)


In [4]:
import os
image_folder = '102flowersSegmentations/segmim'

image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.jpg')])

df_images = pd.DataFrame(image_files, columns=['image_filename'])
df_images['image_id'] = df_images['image_filename'].str.extract(r'(\d+)').astype(int)

df_images

,image_filename,image_id
0,segmim_00001.jpg,1
1,segmim_00002.jpg,2
2,segmim_00003.jpg,3
3,segmim_00004.jpg,4
4,segmim_00005.jpg,5
...,...,...
8184,segmim_08185.jpg,8185
8185,segmim_08186.jpg,8186
8186,segmim_08187.jpg,8187
8187,segmim_08188.jpg,8188


In [5]:
df_image_labels = df_images.merge(df_labels, on='image_id', how='left')

df_image_labels

,image_filename,image_id,labels
0,segmim_00001.jpg,1,77
1,segmim_00002.jpg,2,77
2,segmim_00003.jpg,3,77
3,segmim_00004.jpg,4,77
4,segmim_00005.jpg,5,77
...,...,...,...
8184,segmim_08185.jpg,8185,62
8185,segmim_08186.jpg,8186,62
8186,segmim_08187.jpg,8187,62
8187,segmim_08188.jpg,8188,62


In [6]:
# train, validation and test split
for id in df_setid_trnid[0]:
    df_image_labels.loc[df_image_labels['image_id'] == id, 'set'] = 'train'

for id in df_setid_valid[0]:
    df_image_labels.loc[df_image_labels['image_id'] == id, 'set'] = 'valid'

for id in df_setid_tstid[0]:
    df_image_labels.loc[df_image_labels['image_id'] == id, 'set'] = 'test'

df_image_labels['set'].value_counts()


set
test     6149
valid    1020
train    1020
Name: count, dtype: int64

In [7]:
num_classes = df_image_labels['labels'].nunique()
print("Number of classes: ", num_classes)

# get number of elements in each class
# for i in range(1, num_classes+1):
#     print(f"Class {i}: {df_image_labels[df_image_labels['labels'] == i].shape[0]}")

# put information in a df
df_train_distribution = df_image_labels['labels'].value_counts().reset_index()
df_train_distribution.columns = ['class', 'count']
# sort by class from 1 to 102
df_train_distribution = df_train_distribution.sort_values('count', ascending=False)


df_train_distribution




Number of classes:  102


,class,count
0,51,258
1,77,251
2,46,196
3,73,194
4,89,184
...,...,...
97,7,40
98,45,40
99,34,40
100,1,40


In [8]:
from torchvision.datasets import ImageFolder
test_folder = '102flowers_segmen_split/test'
test_dataset = ImageFolder(test_folder)

In [16]:
import torch
from torchvision import models
import torch.nn as nn

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Step 1: Initialize the ResNet18 model architecture
model = models.resnet18(pretrained=False)  # Set pretrained to False
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 102)  # Adjust the final layer for 102 classes

# Move model to device
model = model.to(device)

# Step 2: Load the saved model weights
model.load_state_dict(torch.load('model_restnet18(3).pth'))

# Set the model to evaluation mode
model.eval()

print("ResNet18 model loaded successfully from 'model_resnet18.pth'")


import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define transformations for the test dataset
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet18 expects 224x224 input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Standard normalization for ResNet
])

# Load the test dataset
test_folder = '102flowers_segmen_split/test'
test_dataset = ImageFolder(test_folder, transform=test_transforms)

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Set the model to evaluation mode
model.eval()

# Initialize variables for accuracy calculation
correct = 0
total = 0

# Disable gradient computation for inference
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Get predicted classes
        _, preds = torch.max(outputs, 1)

        # Update the total and correct predictions
        total += labels.size(0)
        correct += (preds == labels).sum().item()

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Test Accuracy Resnet18: {accuracy:.2f}%')


ResNet18 model loaded successfully from 'model_resnet18.pth'
Test Accuracy Resnet18: 50.50%


In [17]:
import torch
from torchvision import models
import torch.nn as nn

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Step 1: Initialize the ResNet18 model architecture
model = models.vgg16(pretrained=False)  # Set pretrained to False
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 102)  # Adjust the final layer for 102 classes

# Move model to device
model = model.to(device)

# Step 2: Load the saved model weights
model.load_state_dict(torch.load('model_vgg16(8).pth'))

# Set the model to evaluation mode
model.eval()

print("VGG16 model loaded successfully from 'model_vgg16.pth'") 
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define transformations for the test dataset
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet18 expects 224x224 input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Standard normalization for ResNet
])

# Load the test dataset
test_folder = '102flowers_segmen_split/test'
test_dataset = ImageFolder(test_folder, transform=test_transforms)

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Set the model to evaluation mode
model.eval()

# Initialize variables for accuracy calculation
correct = 0
total = 0

# Disable gradient computation for inference
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Get predicted classes
        _, preds = torch.max(outputs, 1)

        # Update the total and correct predictions
        total += labels.size(0)
        correct += (preds == labels).sum().item()

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Test Accuracy VGG16: {accuracy:.2f}%')




c:\Users\alici\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\alici\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


VGG16 model loaded successfully from 'model_vgg16.pth'
Test Accuracy VGG16: 75.00%


In [19]:
model = models.vgg19(pretrained=False)  # Set pretrained to False
num_features = model.classifier[6].in_features

model.classifier[6] = nn.Linear(num_features, 102)  # Adjust the final layer for 102 classes

# Move model to device
model = model.to(device)

# Step 2: Load the saved model weights
model.load_state_dict(torch.load('model_vgg19(14).pth'))

# Set the model to evaluation mode
model.eval()

print("VGG19 model loaded successfully from 'model_vgg19.pth'")

import torch
from torchvision import transforms

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define transformations for the test dataset

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG19 expects 224x224 input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Standard normalization for VGG19
])

# Load the test dataset
test_folder = '102flowers_segmen_split/test'
test_dataset = ImageFolder(test_folder, transform=test_transforms)

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Set the model to evaluation mode
model.eval()

# Initialize variables for accuracy calculation
correct = 0
total = 0

# Disable gradient computation for inference
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Get predicted classes
        _, preds = torch.max(outputs, 1)

        # Update the total and correct predictions
        total += labels.size(0)
        correct += (preds == labels).sum().item()

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Test Accuracy VGG19: {accuracy:.2f}%')


VGG19 model loaded successfully from 'model_vgg19.pth'
Test Accuracy VGG19: 71.91%
